In [ ]:
# Importacion de librerias 
import pandas as pd
import numpy as np
import re
import glob

In [ ]:
#pip install openpyxl

In [ ]:
# 1.  LIMPIEZA DE DATOS // MOROSIDAD SEGÚN DÍAS DE INCUMPLIMIENTO POR CAJA MUNICIPAL - 202504

CARPETA_MORA = r"C:\\Users\\HP\\Desktop\\PORTAFOLIO\DATA\\CAJAS MUNI\\MORA PERIODO"
ruta_mora = glob.glob(CARPETA_MORA+r"\C-*.XLS" )

Lista_mora =[]

# Importar datos en formato excel 

for mor in ruta_mora: 

    BD_MORA=pd.read_excel(mor, engine='xlrd')
    # elminar filas con al nenos un NA 
    BD_MORA=BD_MORA.dropna(how="any")
    # Renombrar encabezados 
    BD_MORA.columns=['ENTIDAD', 'MAS DE 30_D', 'MAS DE 60_D', 'MAS DE 90_D', 'MAS DE 120_D', 'MORA SBS']
    # Unir columas 
    BD_MORA["NUEVA COLMUNA"]=BD_MORA["ENTIDAD"].astype(str)+','+BD_MORA['MORA SBS'].astype(str)
    # Elminar colmuna  
    BD_MORA=BD_MORA.drop('ENTIDAD', axis=1)
    # Anular pivot 
    BD_MORA=BD_MORA.melt(id_vars='NUEVA COLMUNA', var_name='DIAS_IMCUMP', value_name="INCUMP")
    # Separar colmunas 
    BD_MORA[['ENTIDAD','MORA SBS']]=BD_MORA['NUEVA COLMUNA'].str.split(',', expand=True)
    # Eliminar Columna 
    BD_MORA=BD_MORA.drop('NUEVA COLMUNA', axis=1)
    
    Captura=re.search(r'-(\w{2}\d{4})\.XLS$',mor, re.IGNORECASE)
    # condicionamos para se agrupe las capturas y se añada como una columna 
    if Captura:
        MES_ANIO=Captura.group(1)
    else:
        'unknown'
    
    BD_MORA['MES ANIO']=MES_ANIO

    Lista_mora.append(BD_MORA)

DF_consolidado_MOR=pd.concat(Lista_mora, ignore_index=True)

#MORA_=DF_consolidado_MOR[(DF_consolidado_MOR['ENTIDAD']=='TOTAL CAJAS MUNICIPALES DE AHORRO Y CRÉDITO')&(DF_consolidado_MOR['MES ANIO']=="ab2025")]
#print(MORA_)

DF_consolidado_MOR.to_csv("C:\\Users\\HP\\Desktop\\PORTAFOLIO\DATA\\CAJAS MUNI\\MORA PERIODO\\DF_consolidado_MOR.csv", index=False)


In [ ]:
# 2.- LIMPIEZA DE DATOS // CRÉDITOS DIRECTOS SEGÚN SITUACIÓN POR CAJA MUNICIPAL - 202504 

Carpeta = r"C:\\Users\\HP\\Desktop\\PORTAFOLIO\DATA\\CAJAS MUNI\\CREDITO DIRECTO"

archivos = glob.glob(Carpeta + r"\C-*.XLS")
#print(archivos)
lsta_DF= []

# Leer el archivo Excel

for archivo in archivos: 
   #print(f"Leyendo archivo: {archivo}")   # muestra el proceso de lectura de cada archivo

    data = pd.read_excel(archivo, engine='xlrd')
    #BD=data.dropna(how='all')
    BD=data.dropna(how="any")

    # renombrar encabezado de datos 
    BD.columns=['ENT', 'VIG_CP', 'VIG_LP', "R_R", 'VEN', "CJ", "TOT"]
    #Elimianr columnas en especifico 
    BD=BD.drop('TOT', axis=1)
    # Eliminar filas en especifico 
    BD=BD[~BD['ENT'].isin(['TOTAL CAJAS MUNICIPALES DE AHORRO Y CRÉDITO', 'TOTAL CAJAS MUNICIPALES'])]

    #Elimianr columnas en especifico 
    BD_final=BD.melt(id_vars=['ENT'], var_name='SITUACION',value_name="CD")

    Captura=re.search(r'-(\w{2}\d{4})\.XLS$',archivo, re.IGNORECASE)

    if Captura: 
        MES_ANIO= Captura.group(1)
    else:
        MES_ANIO='unknown'

    BD_final['MES_ANIO']=MES_ANIO

    # agregar data frame en la lista 

    lsta_DF.append(BD_final)

DF_consolidado_SIT= pd.concat(lsta_DF, ignore_index=True)

# SUMA CONDICIONADA:

#suma_ab_2025= DF_consolidado_SIT.loc[DF_consolidado_SIT['MES_ANIO']=="ab2025", 'CD'].sum()
#print(suma_ab_2025)

DF_consolidado_SIT.to_csv("C:\\Users\\HP\\Desktop\\PORTAFOLIO\DATA\\CAJAS MUNI\\CREDITO DIRECTO\\DF_consolidado_SIT.csv", index=False)


In [ ]:
# 3.- LIMPIEZA DE DATOS//CRÉDITOS DIRECTOS SEGÚN TIPO DE CD Y SITUACIÓN POR CAJA MUNICIPAL-202504

carpeta_tc = r"C:\\Users\\HP\\Desktop\\PORTAFOLIO\DATA\\CAJAS MUNI\\TIPO DE CREDITO"
archivos_tc = glob.glob(carpeta_tc + r"\C-*.XLS")

Lista_tc=[]

for tc in archivos_tc:
   # Leer el archivo Excel
   DATA_CD = pd.read_excel(tc, engine='xlrd')
   # Rellenar filas - abajo, ubicando colmuna por indice 
   DATA_CD.iloc[:,0]=DATA_CD.iloc[:,0].fillna(method='ffill')
   # Crear copia de datraframe 
   DATA_CD2 = DATA_CD.copy()
   # elminar colmunas en blanco
   DATA_CD2=DATA_CD2.dropna(axis=1, how='all')
   # elminar filas con almenos 1 NA 
   DATA_CD2=DATA_CD2.dropna(how='any')
   # elminar colmunas en especifico 1
   DATA_CD2.columns=DATA_CD2.iloc[0]
   DATA_CD2=DATA_CD2[1:]
   DATA_CD2=DATA_CD2.reset_index(drop=True)
   
   DATA_CD2=DATA_CD2.drop(['TOTAL CAJAS MUNICIPALES DE AHORRO Y CRÉDITO','TOTAL CAJAS MUNICIPALES'], axis=1)
   # Unir dos colmunas 
   DATA_CD2['NUEVO COLUM']=DATA_CD2['Tipo de crédito'].astype(str)+','+DATA_CD2['Situación'].astype(str)
   # elminar colmunas, 
   DATA_CD2=DATA_CD2.drop(['Tipo de crédito','Situación'], axis=1)
   # Pivotar 
   DATA_CD3=DATA_CD2.melt(id_vars=['NUEVO COLUM'], var_name='CM',value_name="CD")
   # separar colmunas, delimitado por comas 
   DATA_CD3[['TIPO CD', 'SITUACION']]=DATA_CD3['NUEVO COLUM'].str.split(',', expand=True)
   # elimar colmuna restante unida
   DATA_CD3=DATA_CD3.drop('NUEVO COLUM', axis=1)
   # capatur de cadena a partir del nombre del archivo para genera una fecha 
   Captura=re.search(r'-(\w{2}\d{4})\.XLS$',tc, re.IGNORECASE)
   if Captura: 
      MES_ANIO= Captura.group(1)
   else:
      MES_ANIO='unknown'

   DATA_CD3['MES_ANIO']=MES_ANIO

   Lista_tc.append(DATA_CD3)

DF_consolidado_TC= pd.concat(Lista_tc, ignore_index=True)

#suma_ene_2025= DF_consolidado_TC.loc[DF_consolidado_TC['MES_ANIO']=="en2024", 'CD'].sum()
#print(suma_ene_2025)

DF_consolidado_TC.to_csv("C:\\Users\\HP\\Desktop\\PORTAFOLIO\DATA\\CAJAS MUNI\\TIPO DE CREDITO\\DF_consolidado_TC.csv", index=False)


In [ ]:
# 4.- LIMPIEZA DE DATOS //MOROSIDAD SEGÚN DÍAS DE INCUMPLIMIENTO POR FINANCIERA - 202504

CARPETA_FINANCE = r"C:\\Users\\HP\\Desktop\\PORTAFOLIO\\DATA\\EMPRESA FINACIERAS\\MOROSIDAD"
RUTA_ARCHIVOS= glob.glob(CARPETA_FINANCE+r"\B-*.XLS")
lISTA_ARCHIVOS=[]

for mora in RUTA_ARCHIVOS:
    # Leer el archivo Excel
    BD_MORA= pd.read_excel(mora, engine='openpyxl')
    BD_MORA=BD_MORA.dropna(how='any')
    BD_MORA.columns=['ENTIDAD', 'MAS DE 30_D', 'MAS DE 60_D', 'MAS DE 90_D', 'MAS DE 120_D', 'MORA SBS']
    BD_MORA['NUEVA COLUMNA']= BD_MORA['ENTIDAD'].astype(str)+','+BD_MORA['MORA SBS'].astype(str)
    BD_MORA=BD_MORA.drop(['ENTIDAD', 'MORA SBS'], axis=1)
    BD_MORA=BD_MORA.melt(id_vars='NUEVA COLUMNA', var_name="SITUACION", value_name="INCUMP")
    BD_MORA[['ENTIDAD', 'MORA SBS']]=BD_MORA['NUEVA COLUMNA'].str.split(',', expand=True)
    BD_MORA=BD_MORA.drop('NUEVA COLUMNA', axis=1)
    CAPTURA=re.search(r'-(\w{2}\d{4})\.XLS$', mora, re.IGNORECASE )
    if CAPTURA: 
        MES_ANIO=CAPTURA.group(1)
    else:
        'unknown'

    BD_MORA['MES_ANIO']=MES_ANIO

    lISTA_ARCHIVOS.append(BD_MORA)

DF_consolidado_MORA_FINANCE=pd.concat(lISTA_ARCHIVOS, ignore_index=True)

DF_consolidado_MORA_FINANCE.to_csv("C:\\Users\\HP\\Desktop\\PORTAFOLIO\\DATA\\EMPRESA FINACIERAS\\MOROSIDAD\\DF_consolidado_MORA_FINANC.csv", index=False)


In [ ]:
# 5.  LIMPIEZA DE DATOS // CRÉDITOS DIRECTOS SEGÚN SITUACIÓN POR EMPRESA FINANCIERA- 202504 

CARPETA_SITUACION = r"C:\\Users\\HP\\Desktop\\PORTAFOLIO\\DATA\\EMPRESA FINACIERAS\\SITUACION"
ARCHIVOS_SITUACION=glob.glob(CARPETA_SITUACION+r"\B-*.XLS")
LISTA_SITUACION=[]

for sit in ARCHIVOS_SITUACION:

    # Leer el archivo Excel
    BD_SIT = pd.read_excel(sit, engine='openpyxl')
    BD_SIT=BD_SIT.dropna(how='any')
    BD_SIT.columns=['ENT', 'VIG_CP', 'VIG_LP', "RES","REF", 'VEN', "CJ", "TOT"]
    BD_SIT=BD_SIT[~BD_SIT['ENT'].isin(['TOTAL EMPRESAS FINANCIERAS'])]
    BD_SIT=BD_SIT.drop('TOT', axis=1)
    BD_SIT=BD_SIT.melt(id_vars='ENT', var_name='SITUACION', value_name='CD')
    CAPTURA_FECHA= re.search(r'-(\w{2}\d{4})\.XLS$', sit, re.IGNORECASE)
    if CARPETA_SITUACION:
        MES_ANIO=CAPTURA_FECHA.group(1)
    else:
        'unknown'
    BD_SIT['MES_ANIO']=MES_ANIO

    LISTA_SITUACION.append(BD_SIT)

DF_consolidado_situacion=pd.concat(LISTA_SITUACION, ignore_index=True)

DF_consolidado_situacion.to_csv("C:\\Users\\HP\\Desktop\\PORTAFOLIO\\DATA\\EMPRESA FINACIERAS\\SITUACION\\DF_consolidado_situacion.csv", index=False)


In [ ]:
# 6. LIMPIEZA DE DATOS // CRÉDITOS DIRECTOS SEGÚN TIPO DE CRÉDITO Y SITUACIÓN POR FINANCIERA - 202504

CARPETA_TCD= r"C:\\Users\\HP\\Desktop\\PORTAFOLIO\\DATA\\EMPRESA FINACIERAS\\TIPO DE CREDITO"
ARCHIVOS_TCD=glob.glob(CARPETA_TCD+r"\B-*.XLS")
LIST_TCD=[]

for TCD in ARCHIVOS_TCD:
    BD_TCD=pd.read_excel(TCD, engine='openpyxl')
    BD_TCD=BD_TCD.dropna(how='all')
    BD_TCD=BD_TCD.transpose()
    # Rellenar filas - abajo, ubicando colmuna por indice 
    BD_TCD.iloc[:,3]=BD_TCD.iloc[:,3].fillna(method='ffill')
    #BD_TCD.iloc[:,4]=BD_TCD.iloc[:,4].fillna(method='ffill')
    BD_TCD.iloc[:,1]=BD_TCD.iloc[:,1].fillna(method='ffill')
    BD_TCD['NUEVA COLUMNA']=BD_TCD[4].astype(str)+';'+BD_TCD[6].astype(str)+';'+BD_TCD[1].astype('str')
    BD_TCD=BD_TCD.drop([4,6,1], axis=1)
    # mover ultima columna a la 1ra 
    BD_TCD=BD_TCD[[BD_TCD.columns[-1]]+list(BD_TCD.columns[:-1])]
    BD_TCD=BD_TCD.transpose()
    BD_TCD=BD_TCD.dropna(how='any')
    # asiganr primera fila como encabezado 
    BD_TCD.columns=BD_TCD.iloc[0]
    BD_TCD=BD_TCD.reset_index(drop=True)
    BD_TCD=BD_TCD.drop(index=0)
    BD_TCD=BD_TCD[~BD_TCD.iloc[:,0].isin(['TOTAL EMPRESAS FINANCIERAS'])]
    BD_TCD=BD_TCD.drop(BD_TCD.columns[-1], axis=1)
    FIRST_NAME=BD_TCD.columns[0]
    BD_TCD=BD_TCD.melt(id_vars=FIRST_NAME, var_name='COLUMUNA TCD', value_name='CD')
    BD_TCD[['TIPO CD', 'SITUACION', 'FECHA']]=BD_TCD['COLUMUNA TCD'].str.split(';', expand=True)
    BD_TCD=BD_TCD.drop('COLUMUNA TCD', axis=1)
    BD_TCD.rename(columns={FIRST_NAME: 'ENT'}, inplace=True)
    LIST_TCD.append(BD_TCD)

DF_consolidado_TCD=pd.concat(LIST_TCD, ignore_index=True)

DF_consolidado_TCD=DF_consolidado_TCD.dropna(axis=1, how='any')


DF_consolidado_TCD.to_csv("C:\\Users\\HP\\Desktop\\PORTAFOLIO\\DATA\\EMPRESA FINACIERAS\\TIPO DE CREDITO\\DF_consolidado_TCD.csv")


#DATE_TCD_UNCOS=DF_consolidado_TCD['FECHA'].unique().tolist()

#for DATE in DATE_TCD_UNCOS:

    #suma_fecha= DF_consolidado_TCD.loc[DF_consolidado_TCD['FECHA']==DATE,'CD'].sum()

    #print(suma_fecha, DATE)